# Monitor SQL Statements - DRAFT

SQL statements are at the core of a relational database. A history of the when and how
those statements were executed can be the key solving problems and improving
performance. Understanding which statements were running at any given time can give
a broad group of user roles insight into what was driving the behavior of the database.

For more technically focused DBA's or Developers, if your users complain that the
database was running slowly, you will eventually need to know the work going on at that
time. Even if you determine that the problem was due to locking contention, or a sudden
spike in CPU usage, or that the DB was running out of free log space, you need to know
which statements were running at the same time the problem occurred. 

For business focused Data Administrators, you can also use a complete statement history to measure
data access and resource use by business or application area. You can use that to
charge for use.

Having insight into the statements the database executing right now, can help you
identify runaway queries that are locking up database resources. Understanding which
statements use the most system resources, or that run most often, will let you
understand which statements to focus on for the biggest impact. Having a detailed
history of all the statements that have run on your system lets you:
* Identify applications and users based on SQL
* See exactly which statements need your attention for tuning or optimization
* Allow you to audit actions against the database
* Chargeback individual users, departments or application owners for what they use

The Db2 Data Management Console supports different kinds of statement monitoring. Each has its
own purpose and strength. Understanding how each works and how to use the Db2 Console
to get the most out of each will help you be more successful in determining the
root cause of problems, see opportunities for improvement and even to build custom
systems for audit or charge-back.

## Using the Console and Open APIs
This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

## Running a Sample Statements and Workloads
This Jupyter Notebook demonstrates how to do different kinds of statement monitoring using real workloads. You create data, run SQL statements and learn how to monitor those statements using:'

1. In-Flight Statement Monitoring using In-Memory Metrics
2. Package Cache Statement Monitoring using In-Memory Metrics
3. Individual Statement History Monitoring using Event Monitoring'

But before we can monitor SQL statement or a workload we need to run those statements and workload.

### Import Helper Classes
To set up to run statements and workloads we need a core set of Jupyter Routines. For more information on these classes, see the lab: [Automate Db2 with Open Console Services](http://localhost:8888/notebooks/Db2_Data_Management_Console_Overview.ipynb)

In [1]:
%run refresh.ipynb
%run ./dmc_workload_setup.ipynb
Console  = 'http://localhost:11080'
Db2Console = Db2(Console+'/dbapi/v4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
profile = 'SAMPLE'
script = 'SELECT TABNAME FROM SYSCAT.TABLES; SELECT INDNAME FROM SYSCAT.INDEXES'
user = 'DB2INST1'
password = 'db2inst1'
rowsReturnedLimit = 5
json = Db2Console.runSQLScript(profile, user, password, script, rowsReturnedLimit)
Db2Console.displayScriptResults(json)

In [ ]:
profileList = ['SAMPLE','HISTORY']
user = 'DB2INST1'
password = 'db2inst1'
script1 = 'SELECT TABNAME FROM SYSCAT.TABLES; SELECT INDNAME FROM SYSCAT.INDEXES'
script2 = 'SELECT * FROM SYSCAT.TABLES; SELECT * FROM SYSCAT.INDEXES'
scriptList = [script1, script2]
profileReps = 2
scriptReps = 2

df = Db2Console.batchScript(profileList, scriptList, user, password, profileReps, scriptReps)
display(df.head(5))

In [ ]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
Db2Console.authenticate(user, password, profile)
r = Db2Console.getCurrentPackageCacheList("false")
if (Db2Console.getStatusCode(r)==200):
    json = Db2Console.getJSON(r)
    if json['count'] > 0:  
        df = pd.DataFrame(json_normalize(json['resources']))
        df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
        if df.empty == True :
            print("No statements in the package cache")
        else :
            display(df[['stmt_text','stmt_exec_time_ms','stmtid']])
    else: 
        print('No data returned') 
else:
    print(Db2Console.getStatusCode(r))

In [ ]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
topX = 10

print(profile+' Package Cache top: '+str(topX))
Db2Console.getCurrentPackageCacheListDF(profile, user, password, topX)

In [ ]:
profileList = ['SAMPLE']
user = 'DB2INST1'
password = 'db2inst1'
script1 = 'SELECT TABNAME FROM SYSCAT.TABLES; SELECT INDNAME FROM SYSCAT.INDEXES'
script2 = 'SELECT * FROM SYSCAT.TABLES; SELECT * FROM SYSCAT.INDEXES'
scriptList = [script1, script2]
profileReps = 2
scriptReps = 2

df = Db2Console.batchScript(profileList, scriptList, user, password, profileReps, scriptReps)
print(profile+' Package Cache top 10')
Db2Console.getCurrentPackageCacheListDF(profile, user, password)

In [ ]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
endTime = int(time.time())*1000
oneHour = 3600000
startTime = endTime - (oneHour *12)
isAverage = 'true' ## false returns Total Runtime
topX = 10

print(profile+' Package Cache top: '+str(topX)+' by average runtime')
Db2Console.getPackageCacheStatementHistory(profile, user, password, startTime, endTime, isAverage)

### Analyze Results
Now we can use the results in the dataframe to look at the results statistically. First we can see the average runtime for each statement across the databases.

In [ ]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtMean = df.groupby(['statement']).mean()
print(stmtMean)

We can also display the total runtime for each statement across databases.

In [ ]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

In [ ]:
longRunningQuery = \
'''
select SYSIBM.SYSTABLES.* from 
SYSIBM.SYSTABLES,
SYSIBM  .SYSCOLUMNS,
SYSIBM  .SYSINDEXES,
SYSIBM  .SYSVIEWS,
SYSIBM  .SYSVIEWDEP,
SYSIBM  .SYSPLAN,
SYSIBM  .SYSPLANDEP,
SYSIBM  .SYSSECTION,
SYSIBM  .SYSSTMT,
SYSIBM  .SYSPLANAUTH,
SYSIBM  .SYSTABAUTH,
SYSIBM  .SYSINDEXAUTH,
SYSIBM  .SYSRELS,
SYSIBM  .SYSROUTINES,
SYSIBM  .SYSROUTINEPARMS,
SYSIBM  .SYSTABCONST,
SYSIBM  .SYSKEYCOLUSE,
SYSIBM  .SYSCONSTDEP
'''

profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
json = Db2Console.runSQLScript(profile, user, password, longRunningQuery)
Db2Console.displayScriptResults(json)

In [ ]:
URL = Console+'/console/?mode=compact#monitor/inflight_executions?profile=SAMPLE'
print(URL)
IFrame(URL, width=1400, height=500)

In [ ]:
http://localhost:11080/dbapi/v4/metrics/statements/inflight_executions/current/list?include_sys=false&limit=100&offset=0&sort=-entry_time
http://localhost:11080/dbapi/v4/metrics/applications/23603/uow/1/inflight_executions/1

In [ ]:
def addEmployeesStatement(startingENO, addENO) :
    addMoreRows = \
    'INSERT INTO EMPLOYEES ' \
    'WITH DT(ENO) AS (VALUES('+str(startingENO)+') UNION ALL SELECT ENO+1 FROM DT WHERE ENO < '+str(startingENO+addENO-1)+')' \
    '''
        SELECT ENO,
        RAND() * 500,
        TRANSLATE(CHAR(INTEGER(RAND()+500000)),
        CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
            WHEN 1 THEN 'aeiou' || 'hjklm'
            WHEN 2 THEN 'aeiou' || 'npqrs'
            WHEN 3 THEN 'fredr' || 'annab'
            ELSE 'aeiou' || 'twxyz' END,
            '1234567890') AS LASTNAME,
       CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
       INTEGER(1000*RAND()*200) AS SALARY
       FROM DT;
    UPDATE  DEPARTMENTS SET REVENUE = REVENUE + INT(100*RAND()*50);    
    '''
    return addMoreRows
print("Defined Add More Rows String Routine")

In [ ]:
def maxEmployees(profile) :
    json = Db2Console.runSQLScript(profile, user, password, 'SELECT MAX(ENO) AS COUNT FROM EMPLOYEES')
    return json['results'][0]['rows'][0][0]

profileList = ['SAMPLE','HISTORY']
for profile in profileList :
    print(profile + ": " + maxEmployees(profile))

In [ ]:
def addEmployees(profile, rows) :
    json = Db2Console.runSQLScript(profile, user, password, addEmployeesStatement(int(maxEmployees(profile))+1, addRows))
    while 'errors' in json:
        print('    * Trying again *')
        json = Db2Console.runSQLScript(profile, user, password, addEmployees(int(maxEmployees(profile))+1, addRows))
    print(str(json['results'][0]['rows_affected']) + ' rows added to ' + profile)

In [ ]:
sqlScriptWorkload1 = \
'''
WITH SALARYBY (DEPTNO, TOTAL) AS
    (SELECT DEPT.DEPTNO DNO, SUM(BIGINT(EMP.SALARY)) TOTAL_SALARY
        FROM EMPLOYEES EMP, DEPARTMENTS DEPT
        WHERE DEPT.DEPTNO = EMP.DEPTNO AND EMP.SALARY > 190000
        GROUP BY DEPT.DEPTNO
        ORDER BY DNO)
SELECT DEPT.DEPTNAME NAME, SALARYBY.TOTAL COST, DEPT.REVENUE, DEPT.REVENUE-SALARYBY.TOTAL PROFIT
FROM SALARYBY, DEPARTMENTS DEPT
WHERE DEPT.DEPTNO = SALARYBY.DEPTNO
AND REVENUE > TOTAL
ORDER BY PROFIT
'''
print("Defined Workload 1 Script")

In [ ]:
sqlScriptWorkload2 = \
'''
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO 
      AND EMP.SALARY < 50000
      AND YEAR(EMP.HIREDATA) > 2010
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO 
      AND EMP.SALARY < 190000
      AND YEAR(EMP.HIREDATA) > 2010
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO, DEPT.REVENUE
  FROM DEPARTMENTS DEPT WHERE DEPT.REVENUE > 450000000;
'''
print("Defined Workload 2 Script")

In [ ]:
import logging
import threading
import time
import concurrent.futures

def thread_function(name):
    logging.info("Thread %s: starting", name)
    profileList = ['SAMPLE','HISTORY']
    user = 'DB2INST1'
    password = 'db2inst1'
    scriptList = [sqlScriptWorkload1, sqlScriptWorkload2]
    profileReps = 2
    scriptReps = 2

    df = Db2Console.batchScript(profileList, scriptList, user, password, profileReps, scriptReps)
    display(df.head(10))
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        executor.map(thread_function, range(2))

We can even graph the total run time for all the statements can compare database performance. Since there are more rows in the employees table in the SAMPLE database it takes longer for the queries to run.

In [2]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
CREATE BUFFERPOOL CONSOLEPOOL ALL DBPARTITIONNUMS SIZE 1000 AUTOMATIC PAGESIZE 32768;
CREATE DATABASE PARTITION GROUP CONSOLEGROUP ON ALL DBPARTITIONNUMS;
CREATE TABLESPACE TS4MONITOR IN CONSOLEGROUP PAGESIZE 32768 MANAGED BY AUTOMATIC STORAGE AUTORESIZE YES INITIALSIZE 100M MAXSIZE 2G BUFFERPOOL CONSOLEPOOL;
CREATE TEMPORARY TABLESPACE TEMPSPACE2 PAGESIZE 32K MANAGED BY AUTOMATIC STORAGE EXTENTSIZE 4 BUFFERPOOL CONSOLEPOOL;
</pre>
</div>

In [ ]:
def createEventMonitorSpace(profile, user, password) :
    createEventMonitorSpace = \
    '''
    CREATE BUFFERPOOL CONSOLEPOOL ALL DBPARTITIONNUMS SIZE 1000 AUTOMATIC PAGESIZE 32768;
    CREATE DATABASE PARTITION GROUP CONSOLEGROUP ON ALL DBPARTITIONNUMS;
    CREATE TABLESPACE TS4MONITOR IN CONSOLEGROUP PAGESIZE 32768 MANAGED BY AUTOMATIC STORAGE AUTORESIZE YES INITIALSIZE 100M MAXSIZE 2G BUFFERPOOL CONSOLEPOOL;
    CREATE TEMPORARY TABLESPACE TEMPSPACE2 PAGESIZE 32K MANAGED BY AUTOMATIC STORAGE EXTENTSIZE 4 BUFFERPOOL CONSOLEPOOL;
    '''
    json = Db2Console.runSQLScript(profile, user, password, createEventMonitorSpace)
    Db2Console.displayScriptResults(json)

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
createEventMonitorSpace(profile, user, password)

In [3]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
CREATE WORKLOAD workload APPLNAME('applicationName')
</pre>
</div>

In [ ]:
def createWorkload(profile, user, password, workload, applicationName) :
    stmt = "CREATE WORKLOAD "+workload+" APPLNAME('"+applicationName+"');"
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
workload = 'SQLEDITOR'
enableWorkloadStatementMonitoring(profile, user, password, workload)
enableAggregateMonitoring(profile, user, password, workload)

In [4]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
CREATE WORKLOAD CONSOLE_WORKLOAD APPLNAME ('UC_*');
</pre>
</div>

In [ ]:
def stopCollectionMonitoringSQL(profile, user, password) :
    stmt = "CREATE WORKLOAD CONSOLE_WORKLOAD APPLNAME ('UC_*');"
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [5]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
ALTER WORKLOAD workload DISABLE;
DROP WORKLOAD workload;
</pre>
</div>

In [ ]:
def dropWorkload(profile, user, password, workload) :
    stmt = \
    'ALTER WORKLOAD '+workload+' DISABLE;' \
    'DROP WORKLOAD '+workload+';'
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [6]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
ALTER WORKLOAD workload COLLECT ACTIVITY DATA ON COORDINATOR WITH DETAILS;
</pre>
</div>

In [ ]:
def enableWorkloadStatementMonitoring(profile, user, password, workload) :
    stmt = 'ALTER WORKLOAD '+workload+' COLLECT ACTIVITY DATA ON COORDINATOR WITH DETAILS;'
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [7]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
ALTER WORKLOAD workload COLLECT AGGREGATE ACTIVITY DATA BASE;
</pre>
</div>

In [ ]:
def enableAggregateMonitoring(profile, user, password, workload) :
    stmt = 'ALTER WORKLOAD '+workload+' COLLECT AGGREGATE ACTIVITY DATA BASE;'
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [8]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
ALTER WORKLOAD workload COLLECT ACTIVITY DATA NONE;
</pre>
</div>

In [ ]:
def disableWorkloadStatementMonitoring(profile, user, password, workload) :
    stmt = 'ALTER WORKLOAD '+workload+' COLLECT ACTIVITY DATA NONE;'
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [9]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
ALTER WORKLOAD workload COLLECT AGGREGATE ACTIVITY DATA NONE;
</pre>
</div>

In [ ]:
def disableAggregateMonitoring(profile, user, password, workload) :
    stmt = 'ALTER WORKLOAD '+workload+' COLLECT AGGREGATE ACTIVITY DATA NONE;'
    Db2Console.displayScriptResults(Db2Console.runSQLScript(profile, user, password, stmt))

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
workload = 'SQLEDITOR'
applicationName = 'SQLEditor'
createWorkload(profile, user, password, workload, applicationName)
enableWorkloadStatementMonitoring(profile, user, password, workload)
enableAggregateMonitoring(profile, user, password, workload)

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
workload = 'SQLEDITOR'
dropWorkload(profile, user, password, workload)

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
workload = 'SQLEDITOR'
disableWorkloadStatementMonitoring(profile, user, password, workload)
disableAggregateMonitoring(profile, user, password, workload)

In [ ]:
profile = 'HISTORY'
user = 'DB2INST1'
password = 'db2inst1'
workload = 'SQLEDITOR'
enableWorkloadStatementMonitoring(profile, user, password, workload)
enableAggregateMonitoring(profile, user, password, workload)

In [ ]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
stopCollectionMonitoringSQL(profile, user, password)

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]